# Meeting around

For any given pair of uid s, determine when and where they could have met each other as
they moved through the building. Please state your assumptions about what would constitute
a “meeting.” Note that the coordinates can be assumed to be 1 unit = 1 meter.

## Design Rationale

When I'm about to work with Data I usually split the work in two parts: Exploration and Optimisation

### Exploration

I try to get some intuition about the dataset. What is it about? What are features? Is it clean or do I need to pre-process it somehow? 

In terms of tooling, I know two different ecosystems R and Python. I like and know Python better so I will go for it and that's why you are reading this notebook. I could probably do the same job with Rmarkdown but I would be less efficient.

### Optimisation

Once I will get a proper sense of the solution I will implement it in a more optimised way and with a faster language. The technological choice is often influenced by the company policy. If everybody codes in Java it would be Java. Today, this my choice so I will pick Scala because I really like the functional programming paradigm.

> **What is following is the Exploration part**

## Prerequesite

* Python 3 (https://www.python.org/download/releases/3.0/)
* Pandas (http://pandas.pydata.org/)

## Load the libraries

In [32]:
import time
from datetime import datetime, timedelta
import dateutil.parser
import pandas as pd
from math import pow, sqrt

## Load the dataset

In [5]:
df = pd.read_csv('reduced.csv')

In [6]:
# Gives an overview of the dataset
df.describe()

,x,y,floor
count,2.228820e+06,2.228820e+06,2.228820e+06
mean,9.407508e+01,7.510458e+01,2.146474e+00
std,1.926607e+01,1.173587e+01,8.091698e-01
min,4.370297e+01,4.249453e+01,1.000000e+00
25%,8.122764e+01,6.622363e+01,1.000000e+00
50%,1.042032e+02,7.115138e+01,2.000000e+00
75%,1.080260e+02,8.643411e+01,3.000000e+00
max,1.150819e+02,1.027560e+02,3.000000e+00


In [7]:
df.head()

,timestamp,x,y,floor,uid
0,2014-07-19T16:00:06.071Z,103.792110,71.504194,1,600dfbe2
1,2014-07-19T16:00:06.074Z,110.336130,100.682839,1,5e7b40e1
2,2014-07-19T16:00:06.076Z,110.066315,86.488736,1,285d22e4
3,2014-07-19T16:00:06.076Z,103.784990,71.456331,1,74d917a1
4,2014-07-19T16:00:06.076Z,109.094950,92.824487,1,3c3649fb


In [8]:
# number of people in the building
len(df.uid.unique())

12991

In [9]:
print('the dataset starts on {0} and ends on {1}'.format(df.timestamp[0], df.timestamp[df.timestamp.count()-1]))

the dataset starts on 2014-07-19T16:00:06.071Z and ends on 2014-07-20T15:59:58.853Z


In [10]:
unique_floors = df.floor.unique()
print('there are {0} floors {1}'.format(len(unique_floors), unique_floors))

there are 3 floors [1 2 3]


## What is a meeting
I will assume that a meeting could happend if the two people where:
 * Are on the same floor
 * In a distance less than 5 units
 * In a timeframe of more or less 5 minutes

## Helpers

In [28]:
def get_subset(df, a, b, floor = None):
    """
    Returns a subset of DataFrame for user a and user b (data are duplicated to preserve the source)
    :param df: dataset
    :type board: pandas.core.frame.DataFrame
    :param a: a first user id
    :type a: str
    :param b: state of the current game
    :type b: a second user id
    """
    assert a != b, "uids should be different"
    indexing = ((df.uid == a) | (df.uid == b)) if floor is None else (((df.uid == a) | (df.uid == b)) & df.floor == floor)
    return df[indexing].copy()

def get_coord(dp):
    return dp.x, dp.y

def calc_coord_dist(p1, p2):
    """
    Returns the Euclidean distance betwen two pairs of coordinate 
    :param x1: x coordinate of first pair
    :type x1: float
    :param y1: y coordinate of first pair
    :type y1: float
    :param x2: x coordinate of second pair
    :type x2: float
    :param y2: y coordinate of second pair
    :type y2: float
    
    """
    x1, y1 = get_coord(p1)
    x2, y2 = get_coord(p2)
    return sqrt(pow(x2 - x1, 2) + pow(y2 - y1, 2))

def from_iso_to_unix_ts(iso):
    """
    Returns a unix timestamp (time from 'epoch' - 00:00:00 UTC on 1 January 1970 - in seconds)
    :param dt: ISO date
    :type dt: str
    :return: a Unix Timestamp
    """
    return int(time.mktime(dateutil.parser.parse(iso).timetuple()))

## Extract a subset of the dataset for 2 user
On one hand it is helping me to get a better intuition of the dataset. 
On the other hand, it will improve the next operations in terms of space and processing time. N is smaller.
(I know I still have "df" in memory but let's assume it is not ;) )

In [17]:
a = '600dfbe2'
b = '5e7b40e1'

In [26]:
subset = get_subset(df, a, b, floor=1)

In [19]:
# the following assertion must be True
subset.count() == df[df.uid == a].count() + df[df.uid == b].count()

timestamp    True
x            True
y            True
floor        True
uid          True
dtype: bool

## Which kind of problem

I'm classifying this problem as [closest pair of points](https://en.wikipedia.org/wiki/Closest_pair_of_points_problem) type

### Solution 1: Brute-force algorithm

This solution has a time complexity of  O(n2) and space O(n). I'm going modify the algorithm not to find the closest but to exit when the first pair fulfilling the requirements is found.

In [40]:
def solution_bf(sub, user_a, user_b, time_frame, min_distance):
    subset_user_a = sub[sub.uid == user_a]
    subset_user_b = sub[sub.uid == user_b]
    for index_a, dp_a in subset_user_a.iterrows():
        for index_b, dp_b in subset_user_b.iterrows():
            if index_a == index_b:
                continue
            time_delta = abs(from_iso_to_unix_ts(dp_a.timestamp) - from_iso_to_unix_ts(dp_b.timestamp))
            distance = calc_coord_dist(dp_a, dp_b)
            if time_delta < time_frame and distance < min_distance:
                return (get_coord(dp_a), get_coord(dp_b))
                # return True 
    return ()
    # return False

print(solution_bf(subset, a, b, 300, 50))

((103.79211, 71.50419417988532), (110.33613000000001, 100.6828393188978))


This solution works fine for small dataset but its time complexity increase quadratically due to the nested for loop. 

If I could obtain a quick response above it is because I defined a radius of 50 meter/unit.

### Solution 2: Divide and Conquer algorithm

complexity O(nlogn)

In [ ]:
def brute_force():
    pass

def strip_closest(strip, d):
    pass

def find_closest(dps, size):
    size = len(x_sorted)
    if size <= 3:
        return solution_bf(dps, a, b, 300, 10)
    
    # find middle point
    middle = size // 2
    middle_dp = dps.iloc[middle]
    
    dl = find_closest(dps[0:middle])
    dr = find_closest(dps[middle:])
    
    # find smaller distance
    d = min(dl, dr)
    
    # build strip array
    strip = []
    for _, dp in dps.iterrows():
        if abs(dp.x - middle_dp.x) < d:
            strip.append(dp)
    
    return min(d, strip_closest(strip, d))
    

def solution_dc(sub):
    x_sorted = sub.copy()
    x_sorted = x_sorted.sort_values(by='x')
    return find_closest(x_sorted)
    